In [2]:
#Here we are preprocessing all the anat and func data. 
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ,get_json_from_file

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

from IPython.display import display, HTML


In [12]:
#Import paths and creation of files for preprocessing
import subprocess
from fsl.wrappers import mcflirt

dataset_id = 'ds000171'
subject = '01'

sample_path = "/home/jovyan/data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)

###################
# Create folders relevant for preprocessing.
# In BIDs, ANYTHING we modify must go in the derivatives folder, to keep original files clean in case we make a mistake.
###################
mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(deriv_root)
mkdir_no_exist(op.join(preproc_root, 'sub-control01'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'fmap'))








path_original_data = os.path.join(bids_root, 'sub-control01/func', 'sub-control01_task-music_run-1_bold.nii.gz')
path_moco_data = os.path.join(preproc_root, 'sub-control01/func', 'sub-control01_task-music_run-1_bold_moco')
mcflirt(infile=path_original_data,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
anatomical_path = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz')
bet_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w')

fmap_path = op.join(bids_root, 'sub-control01', 'func')
subject_dir = 'sub-control{}'.format(subject)


In [4]:

# Import dataset
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", subject_dir # Effectively get all data
               ], check=True)

/home/jovyan/data/dataset/ds000171

👋 Hello! This is openneuro-py 2024.2.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds000171 …


📁 Traversing directories for ds000171 : 18 entities [00:08,  2.05 entities/s]


📥 Retrieving up to 15 files (5 concurrent downloads). 


sub-control01_T1w.nii.gz:   0%|          | 0.00/8.77M [00:00<?, ?B/s]
sub-control01_task-music_run-1_events.tsv: 0.00B [00:00, ?B/s]
sub-control01_T1w.nii.gz:   1%|          | 82.5k/8.77M [00:00<00:19, 462kB/s]
sub-control01_task-music_run-2_events.tsv:   0%|          | 0.00/339 [00:00<?, ?B/s]
sub-control01_T1w.nii.gz:   3%|▎         | 253k/8.77M [00:00<00:08, 1.03MB/s]       
sub-control01_task-music_run-1_bold.nii.gz:   0%|          | 0.00/33.8M [00:00<?, ?B/s]

sub-control01_T1w.nii.gz:   4%|▍         | 372k/8.77M [00:00<00:08, 1.09MB/s]0<?, ?B/s]


sub-control01_task-music_run-3_bold.nii.gz:   0%|          | 0.00/33.9M [00:00<?, ?B/s]
sub-control01_T1w.nii.gz:  10%|▉         | 881k/8.77M [00:00<00:03, 2.51MB/s]00<01:17, 458kB/s]

sub-control01_task-music_run-2_bold.nii.gz:   0%|          | 74.5k/33.9M [00:00<01:23, 425kB/s]



sub-control01_task-music_run-3_events.tsv:   0%|          | 0.00/339 [00:00<?, ?B/s]



sub-control01_T1w.nii.gz:  19%|█▉        | 1.69M/8.77M [00:00<00:01,

✅ Finished downloading ds000171.
 
🧠 Please enjoy your brains.
 


CompletedProcess(args=['openneuro-py', 'download', '--dataset', 'ds000171', '--target-dir', '/home/jovyan/data/dataset/ds000171', '--include', 'sub-control01'], returncode=0)

In [5]:
import shutil

def fsl_anat_wrapped(anatomical_target, output_path):
    os.system('fsl_anat -i {} --clobber --nosubcortseg -o {}'.format(anatomical_target,output_path))
    # Now move all files from the output_path.anat folder created by FSL to 
    # the actual output_path
    fsl_anat_path = output_path+'.anat'
    files_to_move = glob.glob(op.join(fsl_anat_path, '*'))
    for f in files_to_move:
        shutil.move(f, op.join(output_path, op.split(f)[1]))
    
    # Remove the output_path.anat folder
    os.rmdir(fsl_anat_path)

In [10]:
fsl_anat_wrapped(anatomical_path, op.join(preproc_root, 'sub-control01', 'anat'))

Fri Oct 18 18:01:02 UTC 2024
Reorienting to standard orientation
Fri Oct 18 18:01:07 UTC 2024
Automatically cropping the image
Starting Single Image Segmentation
T1-weighted image
Imagesize : 176 x 256 x 170
Pixelsize : 1.2 x 1 x 1

1 5.15907
2 5.58726
3 5.94017
KMeans Iteration 0
KMeans Iteration 1
KMeans Iteration 2
KMeans Iteration 3
KMeans Iteration 4
KMeans Iteration 5
KMeans Iteration 6
KMeans Iteration 7
KMeans Iteration 8
KMeans Iteration 9
KMeans Iteration 10
KMeans Iteration 11
KMeans Iteration 12
KMeans Iteration 13
KMeans Iteration 14
Tanaka Iteration 0 bias field 10
Tanaka-inner-loop-iteration=0 MRFWeightsTotal=2.0141e+07 beta=0.02
Tanaka-inner-loop-iteration=1 MRFWeightsTotal=2.04326e+07 beta=0.02
Tanaka-inner-loop-iteration=2 MRFWeightsTotal=2.04422e+07 beta=0.02
Tanaka-inner-loop-iteration=3 MRFWeightsTotal=2.04426e+07 beta=0.02
Tanaka-inner-loop-iteration=4 MRFWeightsTotal=2.04426e+07 beta=0.02
 CLASS 1 MEAN 100.155 STDDEV 131.718 CLASS 2 MEAN 257.478 STDDEV 69.432 CLA

In [ ]:
# Use glob to find all relevant BOLD files dynamically (e.g., for 'task-music' runs)
func_dir = os.path.join(bids_root, f'sub-control{subject}/func')
bold_files = sorted(glob.glob(os.path.join(func_dir, f'sub-control{subject}_task-music_run-*_bold.nii.gz')))
# Define the white matter segmentation path
white_matter_segmentation = op.join(preproc_root, f'sub-control{subject}', 'anat', 'T1_fast_pve_2.nii.gz')


# Check if any files were found
if not bold_files:
    raise FileNotFoundError(f"No task-music BOLD runs found for subject {subject}.")

# Process each BOLD file
for bold_file in bold_files:
    # Extract the run identifier (e.g., run-1, run-2) from the file path
    run_id = bold_file.split('_')[-2]  # Extracts 'run-1', 'run-2', etc.

    # Create the path for motion-corrected output
    path_moco_data = os.path.join(preproc_root, f'sub-control{subject}/func', f'sub-control{subject}_task-music_{run_id}_bold_moco')

    # Print or use these paths in the rest of your processing pipeline
    print(f'Original Data Path: {bold_file}')
    print(f'Motion Corrected Data Path: {path_moco_data}')
    
    # Now you can add your processing steps (e.g., motion correction, smoothing) here
    # Example motion correction (using FSL's mcflirt):
    os.system(f'mcflirt -in {bold_file} -out {path_moco_data} -refvol 0 -plots')

    # Path to the motion parameters file generated by mcflirt
    mot_params_path = os.path.join(preproc_root, f'sub-control{subject}/func', f'sub-control{subject}_task-music_{run_id}_bold_moco.par')


    #################
    # EPI Registration
    ##################
    
    # Define paths for EPI registration
    epi_target = path_moco_data  # Use the motion corrected data for EPI
    whole_t1 = os.path.join(bids_root, f'sub-control{subject}', 'anat', f'sub-control{subject}_T1w')
    skull_stripped_t1 = os.path.join(bids_root, f'sub-control{subject}', 'anat', f'sub-control{subject}_T1w')
    output_path = os.path.join(preproc_root, f'sub-control{subject}/func', f'sub-control{subject}_task-music_{run_id}_bold_bbr')
    ref_vol_name = os.path.join(preproc_root, f'sub-control{subject}/func', f'sub-control{subject}_task-music_{run_id}_bold_moco_vol_middle')

   
    # Perform smoothing
    smoothed_output_path = '{}_smoothed-6mm'.format(path_moco_data)
    subprocess.run(['fslmaths', output_path, '-s', str(6 / 2.3548), smoothed_output_path])
   


In [26]:
def cleanup_epi_reg(path_to_clean):
    patterns = ['*_fast_*', '*_fieldmap*']
    for p in patterns:
        files = glob.glob(op.join(path_to_clean, p))
        for f in files:
            os.remove(f)

In [ ]:
#cleaning depository : 
cleanup_epi_reg(op.join(preproc_root, 'sub-control01', 'func'))